<a href="https://www.kaggle.com/code/minhcng/sentiment-analysis-using-roberta-lstm?scriptVersionId=137354058" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
!pip install evaluate
!pip install py_vncorenlp
! pip install -U git+https://github.com/huggingface/transformers.git
! pip install -U git+https://github.com/huggingface/accelerate.git


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 18.5 MB/s eta 0:00:0000:010:01
  Created wheel for py_vncorenlp: filename=py_vncorenlp-0.1.4-py3-none-any.whl size=4324 sha256=419bee6c605aad9e1896ae8a3b14e07f5a4d61d978a44b7671e7f4201126013d
  Stored in directory: /root/.cache/pip/wheels/d5/d9/bf/62632cdb007c702a0664091e92a0bb1f18a2fcecbe962d9827
Successfully built py_vncorenlp
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-a2sskig_
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-a2sskig_
  Resolved https://github.com/huggingface/transformers.git to commit 6112b1c6442aaf7affd2b0676a1cd4eee30c45cf
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transf

In [2]:
import py_vncorenlp
py_vncorenlp.download_model(save_dir="./")

VnCoreNLP model folder . already exists! Please load VnCoreNLP from this folder!


In [3]:
!mkdir dataset
!mkdir dataset/training
!mkdir output
!mkdir output/checkpoint
!mkdir output/logging

mkdir: cannot create directory ‘dataset’: File exists
mkdir: cannot create directory ‘dataset/training’: File exists
mkdir: cannot create directory ‘output’: File exists
mkdir: cannot create directory ‘output/checkpoint’: File exists
mkdir: cannot create directory ‘output/logging’: File exists


In [16]:
!cp ../input/sentiment-dataset/*.json dataset/training

In [30]:
import os

CURRENT_PATH = os.getcwd()
TRAINING_DATASET_PATH = f"{CURRENT_PATH}/dataset"
OUTPUT_DIR = f"{CURRENT_PATH}/output"
RAW_DATASET_PATH = "../input/vietnamese-sentiment-analyst/data - data.csv"


# Cleaning Dataset

In [5]:
import pandas as pd
dataset = pd.read_csv(RAW_DATASET_PATH)

In [6]:
dataset.shape

(31460, 4)

In [7]:
dataset.columns

Index(['comment', 'label', 'rate', 'Unnamed: 3'], dtype='object')

In [8]:
dataset.drop(columns=["Unnamed: 3", "rate"], inplace=True, axis=1)
dataset.columns

Index(['comment', 'label'], dtype='object')

In [9]:
dataset["label"].value_counts()

POS    20093
NEG     6669
NEU     4698
Name: label, dtype: int64

In [10]:
dataset=dataset.drop_duplicates("comment")
dataset = dataset.dropna()

In [11]:
dataset["label"].value_counts()

POS    16222
NEG     6330
NEU     4251
Name: label, dtype: int64

In [12]:
dataset["comment"].sample(40)

17130    Sản phẩm không đẹp,bị trầy xướt tùm lùm hết,kh...
17114    Chất lượng sản phẩm tuyệt vời chất lượng đi đô...
6148                                              Hơi mặn.
25175                                                  💗💗.
17666                                Áo đẹp xuất sắc luôn.
17658             Áo bị lỗi mong shop kiểm tra lại kĩ hơn.
7336     Áo đẹp lắm ạ ,chất vải ok , logo và chữ in rất...
11707       Đóng gói sản phẩm rất đẹp và chắc chắn, nhanh.
26065    Thời gian giao hàng rất nhanh Shop phục vụ rất...
6139        Áo đẹp nhưng có điều tay hơi ngắn so với hình.
1237     Hàng ngon, balo handmade đấy các ô (đọc trên c...
5075                          Sao check mã ko ra shop nhỉ.
4567       Form đẹp, shop còn tặng 1 gương mini dễ thương.
14953                     Máy ồn hàng lắp ráp k chắc chắn.
7067     K biết có phải nguyên chất k Chất lượng sản ph...
6980                                 Nhưng màu in hơi lem.
15712    Chất lượng sản phẩm tuyệt vời găng tay đẹp cơ .

In [13]:
import regex as re
import string
import json

emoji_pattern = re.compile("["
                u"\U0001F600-\U0001F64F"  # emoticons
                u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                u"\U0001F680-\U0001F6FF"  # transport & map symbols
                u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                u"\U00002702-\U000027B0"
                u"\U000024C2-\U0001F251"
                u"\U0001f926-\U0001f937"
                u'\U00010000-\U0010ffff'
                u"\u200d"
                u"\u2640-\u2642"
                u"\u2600-\u2B55"
                u"\u23cf"
                u"\u23e9"
                u"\u231a"
                u"\u3030"
                u"\ufe0f"
    "]+", flags=re.UNICODE)

def clean_text(text):
    text = text.lower()
    text = re.sub(emoji_pattern, " ", text)
    text = re.sub(r'([a-z]+?)\1+',r'\1', text)
    text = re.sub(r"(\w)\s*([" + string.punctuation + "])\s*(\w)", r"\1 \2 \3", text)
    text = re.sub(r"(\w)\s*([" + string.punctuation + "])", r"\1 \2", text)
    # text = re.sub(r"(\d)([^\d.])", r"\1 \2", text)
    # text = re.sub(r"([^\d.])(\d)", r"\1 \2", text)
    text = re.sub(f"([{string.punctuation}])([{string.punctuation}])+",r"\1", text)
    text = text.strip()
    while text.endswith(tuple(string.punctuation+string.whitespace)):
        text = text[:-1]
    while text.startswith(tuple(string.punctuation+string.whitespace)):
        text = text[1:]
    text = re.sub(r"\s+", " ", text)
    return text

def map_label(label):
    label_map = {
        "POS": 0,
        "NEG": 1,
        "NEU": 2
    }
    return label_map[label]


In [14]:
dataset["comment"] = dataset["comment"].map(lambda text: clean_text(text))
dataset["label"] = dataset["label"].map(lambda label: map_label(label))

In [15]:
dataset["comment"].sample(40)

676                                     vải áo rất mềm mại
8395     toi k ưng áo màu đỏ bn gửi cho t . đât hai ảo ...
11763        kiến gió nhà mình k ăn , nên k đuổi được kiến
19105    không giống hình , bán xong hỏi shop hok muốn ...
8834                        51 kg mà 2xl vẫn ko kéo lên đc
30532                                              rất ưng
27403                                             điên máu
12840                               form rộng như hình nhé
7171                                               màu đẹp
7914     đã nt yêu cầu shop gửi sớm vì có việc cần nhưn...
15966                          lần sau ủng hộ tiếp dài dài
23822         áo đẹp hơn mong đợi , đường may cung rất đẹp
27560                     áo đẹp . chị chủ shop nhiệt tình
14928                                  chất vải ok mềm mịn
25806    giao k đúng màu , có 1cái giao size nhỏ hơn k ...
21406                                   còn thừa nhiều quá
28128                                            k bám m

In [16]:
dataset=dataset.drop_duplicates("comment")
dataset = dataset.dropna()

In [17]:
dataset = dataset.drop(dataset[dataset["comment"].map(len) < 2].index)

In [18]:
dataset["label"].value_counts()

0    15248
1     6213
2     4138
Name: label, dtype: int64

In [19]:
import random
_dataset = {
    "train": [],
    "eval": [],
    "test": []
}
temp_dataset = [(data["comment"], data["label"]) for _, data in dataset.iterrows()]
random.seed(1234)
random.shuffle(temp_dataset)

In [20]:
temp_dataset[:5]

[('vì mấy đánh giá 5 * mà mình tin tưởng lầm', 1),
 ('đổi sang rêu', 2),
 ('ngon mà', 0),
 ('chất lượng sản phẩm tuyệt vời , đẹp xuất sắc', 0),
 ('áo size s rồi nhưng thật sự còn quá rộng so với mình , chất đẹp , mát , nhân viên phục vụ tốt , giao hàng chậm , chủ shop đáng yêu',
  0)]

In [21]:
split_ratio = {
    "train": 0.8,
    "eval": 0.9,
    "test": 1
}

_i = 0
_len = len(temp_dataset)
for mode in _dataset.keys():
    _dataset[mode] = temp_dataset[_i:int(_len*split_ratio[mode])+1]
    _i = int(_len*split_ratio[mode])+1

In [22]:
from collections import Counter
import json

for mode in _dataset.keys():
    print(len(_dataset[mode]))
    a = Counter([e[1] for e in _dataset[mode]])
    print(a)
    json.dump({"data":_dataset[mode]}, open(f"{TRAINING_DATASET_PATH}/training/{mode}.json", "w", encoding="utf8"), indent=4, ensure_ascii=False)

20480
Counter({0: 12203, 1: 4994, 2: 3283})
2560
Counter({0: 1492, 1: 617, 2: 451})
2559
Counter({0: 1553, 1: 602, 2: 404})


# Training

## Import packages

In [19]:
from transformers import AutoModel, AutoTokenizer, RobertaForSequenceClassification
from transformers import Trainer, SchedulerType
from transformers.training_args import TrainingArguments, OptimizerNames
from transformers.trainer_utils import IntervalStrategy
import torch
import torch.nn as nn
from torch.utils.data import Dataset
import py_vncorenlp
import json
import random

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

## Config

In [20]:
!pip install vncorenlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 25.7 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for vncorenlp: filename=vncorenlp-1.0.3-py3-none-any.whl size=2645951 sha256=09741f6b15ff49fecb84be20875e90a89a285626cc5861b784bdcc3980f0be48
  Stored in directory: /root/.cache/pip/wheels/5d/d9/b3/41f6c6b1ab758561fd4aab55dc0480b9d7a131c6aaa573a3fa
Successfully built vncorenlp


In [46]:
from vncorenlp import VnCoreNLP
class PipelineConfig:
    rdrsegmenter = VnCoreNLP("./VnCoreNLP-1.2.jar", annotators="wseg")

In [47]:
PipelineConfig.rdrsegmenter.tokenize("Tôi là một kỹ sư AI")

[['Tôi', 'là', 'một', 'kỹ_sư', 'AI']]

In [23]:
class SentimentConfig:
    def __init__(
            self,
            pretrained_path: str = None,
            dataset_path: str = None,
            training_output_dir: str = None,
            training_learning_rate: float = None,
            training_weight_decay: float = None,
            training_batch_size: int = None,
            training_num_epochs: int = None,
            training_save_total_limit: int = None,
            training_gradient_accumulation_steps: int = None,
            training_eval_steps: int = None,
            training_logging_steps: int = None,
            training_save_steps: int = None,
            training_logging_dir: str = None,
            training_warm_up_ratio: float = None,
            training_device: str = None,
            training_metrics: str = None,
            segmentor_dir:str = None,
            model_input_max_length:int= None,
            model_num_labels:int=None
    ):
        self.pretrained_path = pretrained_path if pretrained_path is not None else "vinai/phobert-base-v2"
        self.dataset_path = dataset_path if dataset_path is not None else f"{TRAINING_DATASET_PATH}/training"
        self.training_output_dir = training_output_dir if training_output_dir is not None else f"{OUTPUT_DIR}/checkpoint/checkpoint20_1"
        self.training_logging_dir = training_logging_dir if training_logging_dir is not None else f"{OUTPUT_DIR}/logging"
        self.training_learning_rate = training_learning_rate if training_learning_rate is not None else 1.5e-5
        self.training_weight_decay = training_weight_decay if training_weight_decay is not None else 0.01
        self.training_batch_size = training_batch_size if training_batch_size is not None else 32
        self.training_num_epochs = training_num_epochs if training_num_epochs is not None else 10
        self.training_save_total_limit = training_save_total_limit if training_save_total_limit is not None else 3
        self.training_gradient_accumulation_steps = training_gradient_accumulation_steps if training_gradient_accumulation_steps is not None else 2
        self.training_eval_steps = training_eval_steps if training_eval_steps is not None else 200
        self.training_logging_steps = training_logging_steps if training_logging_steps is not None else 200
        self.training_save_steps = training_save_steps if training_save_steps is not None else 200
        self.training_warm_up_ratio = training_warm_up_ratio if training_warm_up_ratio is not None else 0.05
        self.training_device = training_device if training_device is not None else "cuda"
        self.training_metrics = training_metrics if training_metrics is not None else "f1"
        self.model_input_max_length = model_input_max_length if model_input_max_length is not None else 256
        self.model_num_labels = model_num_labels if model_num_labels is not None else 3

## Dataset

In [33]:
random.seed(2023)

class SentimentDataset(Dataset):
    def __init__(self, config, mode, tokenizer):
        super().__init__()
        self.config = config
        self.tokenizer = tokenizer
        self.data = self.load_dataset(path=f"{self.config.dataset_path}/{mode}.json")
        random.shuffle(self.data)
        print(Counter([e[1] for e in self.data]))
        print(f"Load {len(self.data)} examples for {mode} dataset")

    def load_dataset(self, path: str):
        data = json.load(open(path, "r", encoding="utf8"))
        return data["data"]

    def __getitem__(self, idx):
        data_item = self.data[idx]
        labels = torch.zeros(self.config.model_num_labels)
        labels[data_item[-1]] = 1
        text = data_item[0].lower()
        segmented_text = PipelineConfig.rdrsegmenter.tokenize(text)
        tokenized_text = self.tokenizer([" ".join(segmented_text[0])], padding="max_length", truncation=True, max_length=self.config.model_input_max_length, return_tensors="pt")
        return {
            **tokenized_text,
            "labels": labels
        }

    def __len__(self):
        return len(self.data)


## Define Trainer for training model

In [34]:
import os
os.environ["WANDB_DISABLED"] = "true"

### Trainer class

In [35]:
from typing import Optional
from torch.nn import BCEWithLogitsLoss, CrossEntropyLoss, MSELoss
from transformers.modeling_outputs import SequenceClassifierOutput


class SentimentRobertaClassificationHead(nn.Module):
    """Head for sentence-level classification tasks."""

    def __init__(self, config):
        super().__init__()
        self.lstm = nn.LSTM(config.hidden_size, config.hidden_size, 2, bidirectional=True)
        self.dense = nn.Linear(2 * config.hidden_size, config.hidden_size)
        classifier_dropout = (
            config.classifier_dropout if config.classifier_dropout is not None else config.hidden_dropout_prob
        )
        self.dropout = nn.Dropout(classifier_dropout)
        self.out_proj = nn.Linear(config.hidden_size, config.num_labels)

    def forward(self, features, **kwargs):
        # x = features[:, 0, :]  # take <s> token (equiv. to [CLS])
        x = features.mean(dim=1)
        x, (h, c) = self.lstm(x)
        x = self.dropout(x)
        x = self.dense(x)
        x = torch.tanh(x)
        x = self.dropout(x)
        x = self.out_proj(x)
        return x


class SentimentModel(RobertaForSequenceClassification):
  def __init__(self, config):
    super().__init__(config)
    self.classifier = SentimentRobertaClassificationHead(config)

  def forward(
        self,
        input_ids: Optional[torch.LongTensor] = None,
        attention_mask: Optional[torch.FloatTensor] = None,
        token_type_ids: Optional[torch.LongTensor] = None,
        position_ids: Optional[torch.LongTensor] = None,
        head_mask: Optional[torch.FloatTensor] = None,
        inputs_embeds: Optional[torch.FloatTensor] = None,
        labels: Optional[torch.LongTensor] = None,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None,
    ):

      return_dict = return_dict if return_dict is not None else self.config.use_return_dict
      outputs = self.roberta(
          input_ids,
          attention_mask=attention_mask,
          token_type_ids=token_type_ids,
          position_ids=position_ids,
          head_mask=head_mask,
          inputs_embeds=inputs_embeds,
          output_attentions=output_attentions,
          output_hidden_states=output_hidden_states,
          return_dict=return_dict,
      )
      sequence_output = outputs[0]
      logits = self.classifier(sequence_output)

      loss = None
      if labels is not None:
          # move labels to correct device to enable model parallelism
          labels = labels.to(logits.device)
          if self.config.problem_type is None:
              if self.num_labels == 1:
                  self.config.problem_type = "regression"
              elif self.num_labels > 1 and (labels.dtype == torch.long or labels.dtype == torch.int):
                  self.config.problem_type = "single_label_classification"
              else:
                  self.config.problem_type = "multi_label_classification"

          if self.config.problem_type == "regression":
              loss_fct = MSELoss()
              if self.num_labels == 1:
                  loss = loss_fct(logits.squeeze(), labels.squeeze())
              else:
                  loss = loss_fct(logits, labels)
          elif self.config.problem_type == "single_label_classification":
              loss_fct = CrossEntropyLoss()
              loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
          elif self.config.problem_type == "multi_label_classification":
              loss_fct = BCEWithLogitsLoss()
              loss = loss_fct(logits, labels)

      if not return_dict:
          output = (logits,) + outputs[2:]
          return ((loss,) + output) if loss is not None else output

      return SequenceClassifierOutput(
          loss=loss,
          logits=logits,
          hidden_states=outputs.hidden_states,
          attentions=outputs.attentions,
      )

In [36]:
from evaluate import load
from transformers.trainer_utils import EvalPrediction
from collections import Counter


class SentimentTrainer:
    def __init__(self, config: SentimentConfig = None):
        self.config = config if config is not None else SentimentConfig()
        self.model = SentimentModel.from_pretrained(self.config.pretrained_path, num_labels=self.config.model_num_labels)
        self.model.to(self.config.training_device)
        self.tokenizer = AutoTokenizer.from_pretrained(self.config.pretrained_path)
        self.metrics = {
            "precision": load("precision"),
            "recall": load("recall"),
            "f1": load("f1"),
            "accuracy": load("accuracy")
        }
        self.train_dataset, self.eval_dataset = self.load_dataset()

        self.init_trainer()

    def load_dataset(self):
        train_dataset = SentimentDataset(config=self.config, mode="train", tokenizer=self.tokenizer)
        eval_dataset = SentimentDataset(config=self.config, mode="eval", tokenizer=self.tokenizer)
        return train_dataset, eval_dataset

    def collator(self, data):
        batch = {}
        _keys = data[0].keys()
        for key in _keys:
            batch[key] = torch.vstack([ele[key] for ele in data]).to(self.config.training_device)

        return batch

    def compute_metrics(self, eval_predictions: EvalPrediction):
        predictions, labels = eval_predictions
        predictions = torch.from_numpy(predictions).softmax(dim=-1).argmax(dim=-1)
        labels = torch.from_numpy(labels).argmax(dim=-1)

        output =  {
            "precision": self.metrics["precision"].compute(predictions=predictions, references=labels,
                                                           average="weighted")["precision"],
            "recall": self.metrics["recall"].compute(predictions=predictions, references=labels, average="weighted")["recall"],
            "f1": self.metrics["f1"].compute(predictions=predictions, references=labels, average="weighted")["f1"],
            "accuracy": self.metrics["accuracy"].compute(predictions=predictions, references=labels)["accuracy"]
        }
        return output

    def init_trainer(self):
        training_args = TrainingArguments(
            output_dir=self.config.training_output_dir,
            evaluation_strategy=IntervalStrategy.STEPS,
            save_strategy=IntervalStrategy.STEPS,
            per_device_train_batch_size=self.config.training_batch_size,
            per_device_eval_batch_size=self.config.training_batch_size,
            learning_rate=self.config.training_learning_rate,
            weight_decay=self.config.training_weight_decay,
            save_total_limit=self.config.training_save_total_limit,
            num_train_epochs=self.config.training_num_epochs,
            gradient_accumulation_steps=2,
            eval_steps=self.config.training_eval_steps,
            fp16=self.config.training_device == "cuda",
            push_to_hub=False,
            dataloader_num_workers=0,
            logging_dir=self.config.training_logging_dir,
            logging_strategy=IntervalStrategy.STEPS,
            logging_steps=self.config.training_logging_steps,
            save_steps=self.config.training_save_steps,
            logging_first_step=False,
            optim=OptimizerNames.ADAMW_TORCH,
            load_best_model_at_end=True,
            metric_for_best_model=self.config.training_metrics,
            warmup_ratio=self.config.training_warm_up_ratio,
            lr_scheduler_type=SchedulerType.COSINE_WITH_RESTARTS,
            dataloader_pin_memory=False
        )
        self.trainer = Trainer(
            model=self.model,
            args=training_args,
            train_dataset=self.train_dataset,
            eval_dataset=self.eval_dataset,
            tokenizer=self.tokenizer,
            data_collator=self.collator,
            compute_metrics=self.compute_metrics
        )

    def train(self):
        self.trainer.train()

    def eval(self):
        print(self.trainer.evaluate(eval_dataset=self.eval_dataset))


### Training

In [44]:
trainer = SentimentTrainer()

Some weights of SentimentModel were not initialized from the model checkpoint at vinai/phobert-base-v2 and are newly initialized: ['classifier.lstm.bias_hh_l0', 'classifier.lstm.bias_ih_l0', 'classifier.lstm.weight_ih_l0_reverse', 'classifier.lstm.weight_hh_l0', 'classifier.dense.bias', 'classifier.lstm.weight_hh_l1_reverse', 'classifier.out_proj.bias', 'classifier.lstm.bias_hh_l1_reverse', 'classifier.lstm.bias_hh_l0_reverse', 'classifier.out_proj.weight', 'classifier.lstm.bias_ih_l0_reverse', 'classifier.lstm.weight_hh_l0_reverse', 'classifier.lstm.weight_ih_l0', 'classifier.lstm.weight_hh_l1', 'classifier.lstm.bias_ih_l1_reverse', 'classifier.lstm.bias_hh_l1', 'classifier.dense.weight', 'classifier.lstm.bias_ih_l1', 'classifier.lstm.weight_ih_l1', 'classifier.lstm.weight_ih_l1_reverse']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Special tokens have been added in the vocabulary, make sure the associated word embeddin

Counter({2: 13161, 0: 12122, 1: 5075})
Load 30358 examples for train dataset
Counter({2: 2434, 0: 1515, 1: 594})
Load 4543 examples for eval dataset


In [48]:
# bilstm-2, lr=1.5, 
# use mean(output_of_roberta) instead of hidden of first <s> token
trainer.train()

Step,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
200,0.627600,0.526083,0.614780,0.685230,0.643013,0.685230
400,0.371500,0.208361,0.886912,0.879815,0.880892,0.879815
600,0.231300,0.165286,0.902094,0.901387,0.901591,0.901387
800,0.213600,0.155113,0.905874,0.899846,0.900781,0.899846
1000,0.205200,0.152411,0.906155,0.902487,0.903091,0.902487
1200,0.184500,0.152408,0.907138,0.907330,0.907116,0.907330
1400,0.178600,0.147963,0.909081,0.908210,0.908410,0.908210
1600,0.168200,0.159064,0.909456,0.907110,0.907254,0.907110
1800,0.163300,0.153516,0.911560,0.909531,0.909871,0.909531
2000,0.157800,0.160224,0.906501,0.906229,0.906310,0.906229


/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


KeyboardInterrupt: 

# EXPORT model to ONNX

In [49]:
!pip install onnx
!pip install onnxruntime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 23.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 7.5 MB/s eta 0:00:00


In [50]:
import onnx
from transformers import RobertaForSequenceClassification
from onnxruntime.transformers.optimizer import optimize_model
from transformers.convert_graph_to_onnx import quantize
import onnxruntime as ort
import torch
from functools import reduce

In [51]:
best_checkpoint_path = f"{trainer.config.training_output_dir}/checkpoint-1800"
# best_model = RobertaForSequenceClassification.from_pretrained(best_checkpoint_path)
best_model = SentimentModel.from_pretrained(best_checkpoint_path)
best_model.eval()
tokenizer = AutoTokenizer.from_pretrained(best_checkpoint_path)

## Export

### Make saved path

In [52]:
ONNX_FOLDER = f"{trainer.config.training_output_dir}/onnx"
!mkdir "{ONNX_FOLDER}"

### Class for converter

In [53]:
import logging
class ONNXFunc:
    def __init__(self, dummy_inputs, model, input_names, output_names, dynamic_axes):
        self.dummy_inputs = dummy_inputs
        self.model = model
        self.input_names = input_names
        self.output_names = output_names
        self.dynamic_axes = dynamic_axes
        self.logger = logging.getLogger(self.__class__.__name__)

    @staticmethod
    def compose(*functions):
        return reduce(lambda f, g: lambda x: g(f(x)), functions)

    @staticmethod
    def check_onnx_model(path):
        model = onnx.load(path)
        onnx.checker.check_model(model)

    def onnx_only(self, saved_path):
      torch.onnx.export(
          self.model,
          self.dummy_inputs,
          saved_path,
          verbose=True,
          input_names=self.input_names,
          output_names=self.output_names,
          dynamic_axes=self.dynamic_axes
      )
      return saved_path

    def apply_optimized_func(self, saved_path: str):
      use_gpu = trainer.config.training_device=="cuda" and "CUDAExecutionProvider" in ort.get_available_providers()
      optimized_model = optimize_model(saved_path, use_gpu=use_gpu, opt_level=99, verbose=True)
      new_onnx_path = saved_path.replace(".onnx", "_optimized.onnx")
      optimized_model.save_model_to_file(new_onnx_path)
      self.logger.info(f"EXPORT model to ONNX + Optimized - DONE at {new_onnx_path}")
      return new_onnx_path

    def apply_quantized_func(self, saved_path: str):
        from pathlib import Path
        quantized_model_path = quantize(Path(saved_path))
        self.logger.info(f"EXPORT model to ONNX + Quantized - DONE at {quantized_model_path}")
        return quantized_model_path

    @property
    def mapping_func(self):
        return {
            "ONNX": self.onnx_only,
            "optimized_ONNX": self.compose(self.onnx_only, self.apply_optimized_func),
            "quantized_optimized_ONNX": self.compose(
                self.onnx_only, self.apply_optimized_func, self.apply_quantized_func)
        }

    def convert(self, option, saved_path):
        converter = self.mapping_func[option]
        converter(saved_path)

### Convert

In [54]:
import torch
text = ["Tôi là một kĩ sư AI","Tôi là một kĩ sư AI"]
dummy_inputs = tokenizer(text, return_tensors="pt", padding=True)
input_keys = ["input_ids","attention_mask"]
dummy_inputs = tuple(dummy_inputs[i] for i in input_keys)

In [55]:
onnx_conn = ONNXFunc(
    dummy_inputs=dummy_inputs, model=best_model, input_names=["input_ids","attention_mask"], output_names=["logits"], dynamic_axes={
        "input_ids":{
            0: "batch_size",
            1: "dim"
        },
        "attention_mask":{
            0: "batch_size",
            1: "dim"
        },
        "logits": {
            0: "batch_size"
        }
    }
)

In [ ]:
onnx_conn.convert(option="quantized_optimized_ONNX", saved_path=f"{ONNX_FOLDER}/model.onnx")

## Test and compare model

In [ ]:
examples = ["hôm nay tôi vui lắm",
            "hôm nay trời thật đẹp"]
segmented_text = [" ".join(PipelineConfig.rdrsegmenter.word_segment(example)) for example in examples ]
_inputs = tokenizer(examples, return_tensors="pt", padding=True)


### Load dataset for testing

In [ ]:
import json
test_dataset = json.load(open(f"{TRAINING_DATASET_PATH}/training/test.json", "r", encoding="utf8"))
eval_dataset = json.load(open(f"{TRAINING_DATASET_PATH}/training/eval.json", "r", encoding="utf8"))
train_dataset = json.load(open(f"{TRAINING_DATASET_PATH}/training/train.json", "r", encoding="utf8"))
len(test_dataset["data"])

### Init metrics

In [ ]:
from evaluate import load
def custom_metrics(predictions, labels):
    precision = load("precision")
    recall = load("recall")
    f1 = load("f1")
    accuracy = load("accuracy")

    _precision = precision.compute(predictions=predictions, references=labels, average="weighted")
    _recall = recall.compute(predictions=predictions, references=labels, average="weighted")
    _f1 = f1.compute(predictions=predictions, references=labels, average="weighted")
    _accuracy = accuracy.compute(predictions=predictions, references=labels)
    return {
        "precision":_precision,
        "recall":_recall,
        "f1":_f1,
        "accuracy":_accuracy
    }

### Init ONNX sessions

In [ ]:
options = ort.SessionOptions()
# options.intra_op_num_threads = 1
options.graph_optimization_level = ort.GraphOptimizationLevel.ORT_ENABLE_ALL
providers = ["CPUExecutionProvider"]
onnx_path=f"{ONNX_FOLDER}/model.onnx"

In [ ]:
import onnxruntime as ort
ort_session = ort.InferenceSession(onnx_path, providers=providers, sess_options=options)
quantized_model = ort.InferenceSession(f"{ONNX_FOLDER}/model_optimized-quantized.onnx", providers=providers, sess_options=options)

### Test time of sessions with one example

In [ ]:
import time

In [ ]:
t1 = time.time()
best_model(**_inputs)
print("Original model time\t\t", time.time() - t1)

ort_input = {key: _inputs[key].detach().numpy() for key in _inputs.keys() if key!="token_type_ids"}

t2 = time.time()
ort_session.run(["logits"], ort_input)
print("ONNX model time\t\t", time.time() - t2)


t4 = time.time()
quantized_model.run(["logits"], ort_input)
print("optimized + quantized ONNX model time\t\t", time.time() - t4)

### Inference accuracy and time of sessions vs base model

In [ ]:
from tqdm import tqdm
import time
import scipy
import random

In [ ]:
modes = {
    "quantized_opt_onnx":quantized_model,
    "onnx_only": ort_session,
    "base": best_model
}

In [ ]:
import json
_len = len(eval_dataset["data"])
subset = random.choices(eval_dataset["data"], k=_len)
onnx_inputs = [_in.name for _in in quantized_model.get_inputs()]
onnx_outputs = [_out.name for _out in quantized_model.get_outputs()]

for mode, sess in modes.items():
    print(f"PROCESSING {mode}")
    t = time.time()
    acc = 0
    labelsss = []
    predsss = []
    for example, label in tqdm(subset):
        labelsss.append(label)
        segmented_text = [" ".join(PipelineConfig.rdrsegmenter.word_segment(example))]
        _inputs = tokenizer(segmented_text, return_tensors="pt", padding=True)
        if "onnx" in mode:
            ort_input = {key: _inputs[key].detach().numpy() for key in onnx_inputs}
            output = sess.run(onnx_outputs, ort_input)[0]
            output = scipy.special.softmax(output, axis=-1)
            pred = output.argmax(-1).tolist()[0]
        else:
            output = sess(**_inputs)
            output = output.logits.argmax(dim=-1)
            pred = output.item()
        predsss.append(pred)
        if label == pred:
            acc += 1
    end_time=time.time()
    print(f"{mode} DONE with avg time={(end_time-t)/_len}")
    print(json.dumps(custom_metrics(predsss, labelsss), indent=4, ensure_ascii=False))

- Model without ONNX:
    - accuracy = 0.857
    - f1 = 0.856
    - avg time = 158.717 ms/example
- ONNX only:
    - accuracy = 0.857
    - f1 = 0.856
    - time = 58.079 ms/example
- ONNX and quantize:
    - accuracy = 0.858
    - f1 = 0.856
    - avg time = 29.912 ms/example


#### Test quantized model

In [ ]:
def reverse_label(label):
    label_map = {
        0: "POS",
        1: "NEG",
        2: "NEU"
    }
    return label_map[label]

In [ ]:
_len = 1000
subset = random.choices(eval_dataset["data"], k=_len)
acc=0
for example, label in subset:
    segmented_text = [" ".join(PipelineConfig.rdrsegmenter.word_segment(example))]
    _inputs = tokenizer(segmented_text, return_tensors="pt", padding=True)

    ort_input = {key: _inputs[key].detach().numpy() for key in onnx_inputs}
    output = quantized_model.run(onnx_outputs, ort_input)[0]
    output = scipy.special.softmax(output, axis=-1)
    pred = output.argmax(-1).tolist()[0]

    if label == pred:
        acc += 1
    else:
        print("==============")
        print(example)
        print(reverse_label(label), "!=", reverse_label(pred))